In [2]:
from jaref_bot.data.http_api import ExchangeManager, BybitRestAPI, GateIORestAPI
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, coint
from lightweight_charts import Chart, JupyterChart

import pandas as pd
import polars as pl
import polars_ols as pls
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import numpy as np
from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo

from tqdm.notebook import tqdm

from jaref_bot.db.postgres_manager import DBManager
from jaref_bot.config.credentials import host, user, password, db_name
db_params = {'host': host, 'user': user, 'password': password, 'dbname': db_name}
db_manager = DBManager(db_params)

exc_manager = ExchangeManager()
exc_manager.add_market("bybit_linear", BybitRestAPI('linear'))

In [3]:
# df_1 = df_1.with_columns(
#             pl.col("close").pct_change().alias('returns'),
#             pl.col("close").log().diff().alias('log_returns')
#         )

In [4]:
def calculate_profit(open_price, close_price, n_coins=None, usdt_amount=None, side='long', fee_rate=0.0002):
    if n_coins is None:
        n_coins = round(usdt_amount / open_price)
    
    usdt_open = n_coins * open_price
    open_fee = usdt_open * fee_rate

    usdt_close = n_coins * close_price
    close_fee = usdt_close * fee_rate
    # print(f'{open_fee=}; {close_fee=}')

    if side == 'long':
        profit = usdt_close - usdt_open - open_fee - close_fee
    elif side == 'short':
        profit = usdt_open - usdt_close - open_fee - close_fee
    return profit

#### Загрузка данных с ByBit

In [ ]:
# Гиперпараметры модели: roll_wind, std_coef (при каком отклонении от среднего входим в сделку)
download_period = '1h'
grouping_period = '24h'
roll_wind = 60
n_iters = 10
sym_1 = 'STRK_USDT'
sym_2 = 'XAI_USDT'

df_1 = await exc_manager.get_candles(symbol=sym_1, interval=download_period, n_iters=n_iters)
df_1 = df_1[0].sort_index()
df_2 = await exc_manager.get_candles(symbol=sym_2, interval=download_period, n_iters=n_iters)
df_2 = df_2[0].sort_index()

cols = ['Close']
df = df_1[cols].merge(df_2[cols], how='inner', on='Date', suffixes=(f'_{sym_1}', f'_{sym_2}'))
df = df.rename(columns={f'Close_{sym_1}': sym_1, f'Close_{sym_2}': sym_2})

stat_df = make_stat_df(df, sym_1, sym_2, grouping_period, roll_wind)

In [ ]:
stat_df.head(2)

In [ ]:
print_pair(stat_df, sym_1, sym_2)

#### Создание датафрейма с оконными функциями

In [1]:
from jaref_bot.analysis.backtest.pair_trading import make_df_from_orderbooks, make_spread_df_bulk, make_trunc_df, make_spread_df

In [ ]:
token_1 = 'CELO'
token_2 = 'GRT'

start_time = datetime(2025, 8, 23, 18, 0, tzinfo=ZoneInfo("Europe/Moscow"))
valid_time = datetime(2025, 8, 26, 12, 0, tzinfo=ZoneInfo("Europe/Moscow"))
end_time = datetime(2025, 10, 30, 23, 50, tzinfo=ZoneInfo("Europe/Moscow"))

In [ ]:
df_1 = db_manager.get_raw_orderbooks(exchange='bybit',
                                     market_type='linear',
                                     token=token_1 + '_USDT')
df_2 = db_manager.get_raw_orderbooks(exchange='bybit',
                                     market_type='linear',
                                     token=token_2 + '_USDT')

print(f'{token_1}: {df_1['time'].head(1).item()} - {df_1['time'].tail(1).item()}')
print(f'{token_2}: {df_2['time'].head(1).item()} - {df_2['time'].tail(1).item()}')

In [ ]:
df = make_df_from_orderbooks(df_1, df_2, token_1, token_2, start_time=start_time, log_=True)

In [ ]:
df_hour = make_trunc_df(df, timeframe='1h', token_1=token_1, token_2=token_2, method='last')
df_4hour = make_trunc_df(df, timeframe='4h', token_1=token_1, token_2=token_2, method='last')
df_5min = make_trunc_df(df, timeframe='5m', token_1=token_1, token_2=token_2, method='last')
df_sec = make_trunc_df(df, timeframe='1s', token_1=token_1, token_2=token_2, start_date=valid_time, method='last')

In [ ]:
df_sec.head(2)

In [ ]:
%%time
hour4_suffix = '4h'
hour1_suffix = '1h'
minute5_suffix = '5m'

rows_buffer = []

hour4_winds = (8, 10, 12, 16, 20)
hour1_winds = (12, 18, 24, 36, 48, 60)
minute5_winds = (60, 90, 120, 240, 360, 480)

max_hour1_wind = max(hour1_winds)
max_hour4_wind = max(hour4_winds)
max_minute5_wind = max(minute5_winds)

# --- Предварительно вычислим названия столбцов для переименования в теле цикла и столбцов для удаления ---
mean_hour1_cols = ['mean_' + str(c) for c in hour1_winds]
std_hour1_cols = ['std_' + str(c) for c in hour1_winds]
h1_cols_to_drop = mean_hour1_cols + std_hour1_cols

mean_hour4_cols = ['mean_' + str(c) for c in hour4_winds]
std_hour4_cols = ['std_' + str(c) for c in hour4_winds]
h4_cols_to_drop = mean_hour4_cols + std_hour4_cols

mean_minute5_cols = ['mean_' + str(c) for c in minute5_winds]
std_minute5_cols = ['std_' + str(c) for c in minute5_winds]
m5_cols_to_drop = mean_minute5_cols + std_minute5_cols

hour1_rename_cols = {f'z_score_{col}': f'z_score_{col}_{hour1_suffix}' for col in hour1_winds}
hour4_rename_cols = {f'z_score_{col}': f'z_score_{col}_{hour4_suffix}' for col in hour4_winds}
minute5_rename_cols = {f'z_score_{col}': f'z_score_{col}_{minute5_suffix}' for col in minute5_winds}

# --- Основной цикл --- 
try:
    for row in tqdm(df_sec[:].iter_slices(1), total=df_sec.height):
        # --- 4-ЧАСОВОЕ ОКНО ---
        hour4_stat = df_4hour.filter(pl.col('ts') < row['ts']).tail(max_hour4_wind)
        hour4_stat = hour4_stat.vstack(row)
        th4 = make_spread_df_bulk(hour4_stat, token_1, token_2, winds=hour4_winds).tail(1)
        th4 = th4.drop(h4_cols_to_drop)
        th4 = th4.rename(hour4_rename_cols) 
        
        # --- ЧАСОВОЕ ОКНО ---
        hour1_stat = df_hour.filter(pl.col('ts') < row['ts']).tail(max_hour1_wind)
        hour1_stat = hour1_stat.vstack(row)
        th1 = make_spread_df_bulk(hour1_stat, token_1, token_2, winds=hour1_winds).tail(1)
        th1 = th1.drop(h1_cols_to_drop + ['time', 'ts', token_1, token_2, 'spread'])
        th1 = th1.rename(hour1_rename_cols)        
                
        # --- 5-МИНУТНОЕ ОКНО ---
        minute5_stat = df_5min.filter(pl.col('ts') < row['ts']).tail(max_minute5_wind)
        minute5_stat = minute5_stat.vstack(row)
        tm5 = make_spread_df_bulk(minute5_stat, token_1, token_2, winds=minute5_winds).tail(1)
        tm5 = tm5.drop(m5_cols_to_drop + ['time', 'ts', token_1, token_2, 'spread'])
        tm5 = tm5.rename(minute5_rename_cols)
                
        # --- финальная сборка строки результата ---
        curr_row = th4.hstack(th1).hstack(tm5)
        rows_buffer.extend(curr_row.to_dicts())
        
    result_df = pl.DataFrame(rows_buffer, infer_schema_length=None)
    rows_buffer = []

except KeyboardInterrupt:
    result_df = pl.DataFrame(rows_buffer, infer_schema_length=None)
    rows_buffer = []

In [ ]:
result_df.tail(2)

In [ ]:
# 5_000, Wall time: 12.2 s

In [ ]:
method = 'dist'

result_df.write_parquet(f'./data/{token_1}_{token_2}_{method}.parquet')

#### Исследование монет

In [ ]:
from jaref_bot.utils.data import make_price_df_from_orderbooks_bulk, normalize
from jaref_bot.analysis.utils import make_spread_df, make_df_from_orderbooks

In [ ]:
# Если будет нужно отфильтровать датафрейм по времени

start_date = datetime(2025, 8, 23, 18, 0, tzinfo=ZoneInfo("Europe/Moscow"))
# end_date = datetime(2025, 8, 12, 8, 30, tzinfo=ZoneInfo("Europe/Moscow"))
# tdf = spread_df.filter((pl.col('time') > start_date) & (pl.col('time') < end_date))

In [ ]:
# raw_df = db_manager.get_table('raw_orderbook_data', df_type='polars')

In [ ]:
# Задаём названия токенов для анализа
tokens = ['AKT', 'APT', 'ARB', 'ARKM', 'C98',  'CELO', 'CHR', 'ENJ', 'FIL', 'FLOW', 'GALA', 'GMT', 'GRT', 'GTC',
        'MANA', 'OGN', 'ONDO', 'ONG', 'OP', 'PHA', 'ROSE', 'SAND', 'STG', 'SNX', 'VET']
# token_pairs = [f'{t}_USDT' for t in tokens]

exc_manager = ExchangeManager()
exc_manager.add_market("bybit_linear", BybitRestAPI('linear'))
exc_manager.add_market("gate_linear", GateIORestAPI('linear'))
coin_information = exc_manager.get_instrument_data()

In [ ]:
# Создадим список из датафреймов для удобства пакетной обработки, а также получим кол-во знаков после запятой для округления 
dfs = []
token_dp = {}

for token in tokens:
    df_token = db_manager.get_orderbooks(exchange='bybit', market_type='linear', symbol=token + '_USDT', interval='1min', start_date=start_date)
    try:
        dp = len(
            coin_information['bybit_linear'][token + '_USDT']['qty_step']
            .to_eng_string()
            .split('.')[1]
        )
    except IndexError:
        dp = 0

    token_dp[token] = dp
    dfs.append(df_token)

In [ ]:
# Создадим датафрейм, содержащий только цены, для анализа взаимосвязи монет
price_df = make_price_df_from_orderbooks_bulk(dfs=dfs, tokens=tokens, trunc='5m')
price_df.tail(2)

In [ ]:
# Нормируем значения
normed_df = normalize(df=price_df, method='minimax', shift_to_zero=False)
normed_df.tail(3)

In [ ]:
# Нарисуем график с нормализованными ценами
date_col = 'bucket' if 'bucket' in normed_df.columns else 'time'
price_cols = [c for c in normed_df.columns if c != date_col]

plt.figure(figsize=(14, 4))
for col in price_cols:
    plt.plot(normed_df[date_col].to_list(), normed_df[col].to_list(), label=col)

plt.xlabel("Время")
plt.ylabel("Нормализованная цена")
plt.title("Нормализованные цены монет")
# plt.legend()
plt.grid(True)
plt.tight_layout()

In [ ]:
# Посчитаем евклидово расстояние между парами криптовалют
import itertools
import math

results = []
    
# Перебираем все уникальные пары колонок
for col1, col2 in itertools.combinations(price_cols, 2):
    diff_sq = (normed_df[col1] - normed_df[col2]) ** 2
    distance = math.sqrt(diff_sq.sum())
    results.append((col1, col2, distance))

In [ ]:
pl.DataFrame(results, schema=["coin1", "coin2", "dist"], orient="row").sort('dist')[10:20]

In [ ]:
token_1 = 'CELO'
token_2 = 'GRT'

df_1 = db_manager.get_orderbooks(exchange='bybit', market_type='linear', symbol=token_1 + '_USDT', interval='1min')
df_2 = db_manager.get_orderbooks(exchange='bybit', market_type='linear', symbol=token_2 + '_USDT', interval='1min')

In [ ]:
start_time = datetime(2025, 8, 23, 12, 0, tzinfo=ZoneInfo("Europe/Moscow"))
end_time = datetime(2025, 8, 29, 21, 0, tzinfo=ZoneInfo("Europe/Moscow"))

df = make_df_from_orderbooks(df_1, df_2, token_1, token_2, start_time=start_time, end_time=end_time)
cols = [col for col in df.columns if 'ask' in col or 'bid' in col]
df = df.drop(cols)
df = make_spread_df(df, token_1, token_2, wind=480).drop_nulls()
coef = df[0][token_1].item() / df[0][token_2].item()

In [ ]:
# Нарисуем график с нормализованными ценами
date_col = 'bucket' if 'bucket' in df.columns else 'time'
price_cols = [c for c in df.columns if c != date_col]

plt.figure(figsize=(14, 4))
plt.plot(df[date_col], df[token_1], label=token_1);
plt.plot(df[date_col], coef * df[token_2], label=token_2);
plt.xlabel("Время")
plt.ylabel("Цена")
plt.title(f"Приведённые к одному масштабу цены монет. Coef: {coef:.2f}")
plt.legend()
plt.grid(True)
plt.tight_layout()

In [ ]:
std = 2
upper_bound = df['mean'] + std * df['std']
lower_bound = df['mean'] - std * df['std']

plt.figure(figsize=(14, 2))
plt.plot(df[date_col], df['spread']);
plt.plot(df[date_col], upper_bound)
plt.plot(df[date_col], lower_bound)
plt.grid()

In [ ]:
train_time = datetime(2025, 8, 15, 3, 0, tzinfo=ZoneInfo("Europe/Moscow")) # За какое время рассчитываем mean & std для StandartScaler

spr_train = df.filter(pl.col(date_col) <= train_time)['spread'].to_numpy().reshape(-1, 1)
spr_test = df.filter(pl.col(date_col) > train_time)['spread'].to_numpy().reshape(-1, 1)

In [ ]:
scaler = StandardScaler()
scaler.fit(spr_train)
spr_normed = scaler.transform(spr_test).ravel()

In [ ]:
plt.figure(figsize=(14, 2))
plt.plot(spr_normed);
plt.title("Стандартизованный спред")
plt.grid()

#### Симуляция одной торговой пары

In [ ]:
from jaref_bot.analysis.backtest.pair_trading import make_df_from_orderbooks, backtest
from jaref_bot.analysis.strategy_analysis import analyze_strategy

from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo
import polars as pl
from jaref_bot.data.http_api import ExchangeManager, BybitRestAPI, GateIORestAPI
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

from jaref_bot.db.postgres_manager import DBManager
from jaref_bot.config.credentials import host, user, password, db_name
db_params = {'host': host, 'user': user, 'password': password, 'dbname': db_name}
db_manager = DBManager(db_params)

In [ ]:
token_1 = 'ARKM'
token_2 = 'SAND'

start_time = datetime(2025, 8, 26, 12, 0, tzinfo=ZoneInfo("Europe/Moscow"))
valid_time = datetime(2025, 8, 28, 12, 0, tzinfo=ZoneInfo("Europe/Moscow"))
end_time = datetime(2025, 12, 29, 22, 0, tzinfo=ZoneInfo("Europe/Moscow"))

spread_df = pl.read_parquet(f'./data/{token_1}_{token_2}_dist.parquet')

In [ ]:
spread_df.tail(2)

In [ ]:
df_1 = db_manager.get_raw_orderbooks(exchange='bybit',
                                     market_type='linear',
                                     token=token_1 + '_USDT',
                                     start_time=start_time,
                                     end_time=end_time)
df_1 = df_1.with_columns(pl.col('time').dt.epoch('s').alias('ts'))
df_2 = db_manager.get_raw_orderbooks(exchange='bybit',
                                     market_type='linear',
                                     token=token_2 + '_USDT',
                                     start_time=start_time,
                                     end_time=end_time)
df_2 = df_2.with_columns(pl.col('time').dt.epoch('s').alias('ts'))

bid_ask_df = make_df_from_orderbooks(df_1, df_2, token_1, token_2, start_time, end_time)
bid_ask_df = bid_ask_df.select('ts', f'{token_1}_bid_price', 
                               f'{token_1}_ask_price', 
                               f'{token_2}_bid_price', 
                               f'{token_2}_ask_price'
                              )

In [ ]:
bid_ask_df.tail(2)

In [ ]:
# Загружаем с биржи ByBit техническую информацию по монетам (шаг цены, округление цены в usdt etc.)
exc_manager = ExchangeManager()
exc_manager.add_market("bybit_linear", BybitRestAPI('linear'))
coin_information = exc_manager.get_instrument_data()

# Сохраним информацию о шаге цены монет в переменных
dp_1 = float(coin_information['bybit_linear'][token_1 + '_USDT']['qty_step'])
ps_1 = int(coin_information['bybit_linear'][token_1 + '_USDT']['price_scale'])
dp_2 = float(coin_information['bybit_linear'][token_2 + '_USDT']['qty_step'])
ps_2 = int(coin_information['bybit_linear'][token_2 + '_USDT']['price_scale'])

In [ ]:
search_space = (
            ('4h', 4), ('4h', 6), ('4h', 8), ('4h', 10),
            ('4h', 12),
            ('1h', 6), ('1h', 8), ('1h', 12), ('1h', 24),
            ('1h', 36), ('1h', 48),
            ('15m', 10), ('15m', 20), ('15m', 30), ('15m', 40), ('15m', 50),
            ('15m', 60), ('15m', 80), ('15m', 100), ('15m', 120), ('15m', 160),
            ('5m', 20), ('5m', 30), ('5m', 45), ('5m', 60), ('5m', 90),
            ('5m', 120), ('5m', 240), ('5m', 360)
        )

In [ ]:
for tf, wind in search_space:
    col_name = f'z_score_{wind}_{tf}'
    df = spread_df.select('time', 'ts', 'spread', col_name)
    df = df.rename({col_name: 'z_score'})
    df = df.join(bid_ask_df, on='ts').filter((pl.col('time') > start_time) & (pl.col('time') < valid_time))

    plt.figure(figsize=(15, 3))
    plt.title(f'{tf}; {wind}')
    plt.plot(df['z_score'])
    plt.grid()
    plt.show()

In [ ]:
tf = '4h' # таймфрейм для агрегации цен
wind = 15 # размер окна для оконных функций

col_name = f'z_score_{wind}_{tf}'
df = spread_df.select('time', 'ts', 'spread', col_name).filter((pl.col('time') > start_time) & (pl.col('time') < end_time))
df = df.rename({col_name: 'z_score'})
df = df.join(bid_ask_df, on='ts')

In [ ]:
plt.figure(figsize=(15, 3))
plt.title(f'{tf}; {wind}')
plt.plot(df['z_score'])
plt.grid()

In [ ]:
full_df = spread_df.select('time', 'ts', 'spread', col_name)
full_df = full_df.rename({col_name: 'z_score'})
full_df = full_df.join(bid_ask_df, on='ts')

valid_df = spread_df.select('time', 'ts', 'spread', col_name).filter((pl.col('time') > start_time) & (pl.col('time') < valid_time))
valid_df = valid_df.rename({col_name: 'z_score'})
valid_df = valid_df.join(bid_ask_df, on='ts')

test_df = spread_df.select('time', 'ts', 'spread', col_name).filter((pl.col('time') > valid_time) & (pl.col('time') < end_time))
test_df = test_df.rename({col_name: 'z_score'})
test_df = test_df.join(bid_ask_df, on='ts')

In [ ]:
# %%timeit
mode = 'full' # test / valid / full
leverage = 1

if mode == 'valid':
    df = valid_df
    time_1 = start_time
    time_2 = valid_time
elif mode == 'test':
    df = test_df
    time_1 = valid_time
    time_2 = end_time
elif mode == 'full':
    df = full_df
    time_1 = start_time
    time_2 = end_time

params = {'low_in': -1.0, 'high_in': 2.6, 'low_out': -2.4, 'high_out': 1.0}

trades_df = backtest(df, token_1, token_2, dp_1, dp_2, ps_1, ps_2, 
            thresh_low_in=params['low_in'], thresh_high_in=params['high_in'], 
            thresh_low_out=params['low_out'], thresh_high_out=params['high_out'], 
            long_possible=True, short_possible=True,
            method_in='direct', method_out='direct',
            balance=200, order_size=100, fee_rate=0.00055, stop_loss_std=5.0, sl_method='leave',
            sl_seconds = 60,
            leverage=leverage,
            verbose=2)

In [ ]:
trades_df.tail()

In [ ]:
metrics = analyze_strategy(trades_df, start_date=time_1, end_date=time_2, initial_balance=200.0)

In [ ]:
metrics

In [ ]:
# 0. Сделать проверку доступного объёма.
# 1. Проверить для разных способов вычисления скользящих значений (по цене закрытия или другими способами)
# 2. Проверить разные способы распределения денег между плечами ордера
# 3. Проверить, как изменится доходность, если размер ставки увеличивать с ростом банкролла

In [ ]:
time_1 = datetime(2025, 8, 26, 12, 00, 42, tzinfo=ZoneInfo("Europe/Moscow"))
time_2 = datetime(2025, 8, 27, 21, 12, 56, tzinfo=ZoneInfo("Europe/Moscow"))
df.filter((pl.col('time') > time_1) & (pl.col('time') < time_2)).drop('ts')

In [ ]:
y = df['close_1']
X = df['close_2']
X = sm.add_constant(X)  # добавляем константу (alpha)

model = sm.OLS(y, X).fit()
alpha, beta = model.params
print(f"alpha = {alpha:.4f}, beta = {beta:.4f}")

residuals = y - (alpha + beta * df['close_2'])
adf_result = adfuller(residuals)
p_value = adf_result[1]
print(f'{p_value=:.2f}')

In [ ]:
(df['close_1'] - (alpha + beta * df['close_2'])).plot(figsize=(14, 2));

#### Trading

In [ ]:
from jaref_bot.data.http_api import ExchangeManager, BybitRestAPI, GateIORestAPI

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import pandas as pd
import polars as pl
import numpy as np
from datetime import datetime

from jaref_bot.db.postgres_manager import DBManager
from jaref_bot.db.redis_manager import RedisManager
from jaref_bot.config.credentials import host, user, password, db_name

db_params = {'host': host, 'user': user, 'password': password, 'dbname': db_name}
postgre_manager = DBManager(db_params)

redis_price_manager = RedisManager(db_name = 'orderbooks')
redis_order_manager = RedisManager(db_name = 'orders')

exc_manager = ExchangeManager()
exc_manager.add_market("bybit_linear", BybitRestAPI('linear'))
# exc_manager.add_market("okx_linear", OKXRestAPI('linear'))
# exc_manager.add_market("gate_linear", GateIORestAPI('linear'))

In [ ]:
exc = 'bybit_linear'
interval = '1h'
n_iters = 1
token_1 = 'STRK'
token_2 = 'XAI'

res_1 = await exc_manager.get_candles(symbol=f'{token_1}_USDT', interval=interval, n_iters=n_iters)
res_2 = await exc_manager.get_candles(symbol=f'{token_2}_USDT', interval=interval, n_iters=n_iters)
res_1 = res_1[exc]
res_2 = res_2[exc]

In [ ]:
hist_df = res_1[['Close']].merge(res_2[['Close']], how='inner', on='Date', suffixes=(f'_{token_1}', f'_{token_2}')
                      ).rename(columns={f'Close_{token_1}': token_1, f'Close_{token_2}': token_2}).reset_index()
hist_df = pl.DataFrame(hist_df)
curr_time_bar = hist_df[-1, 'Date'].hour

In [ ]:
hist_df.tail(2)

In [ ]:
current_data = redis_price_manager.get_orderbooks(1)
t_1 = current_data.filter(pl.col('symbol') == f'{token_1}_USDT').select('bidprice_0', 'askprice_0').mean_horizontal().item()
t_2 = current_data.filter(pl.col('symbol') == f'{token_2}_USDT').select('bidprice_0', 'askprice_0').mean_horizontal().item()

Moscow_TZ = timezone(timedelta(hours=3))
ct = datetime.now(Moscow_TZ).replace(microsecond=0)

new_row = pl.DataFrame({
    "Date": [ct]
}).with_columns(
    pl.col("Date").cast(pl.Datetime("ns", "Europe/Moscow"))
).with_columns([
    pl.lit(t_1).alias(token_1),
    pl.lit(t_2).alias(token_2)
])

df = hist_df.vstack(new_row)

wind = 4

df = df.with_columns(
    (pl.col(f'{token_1}') / pl.col(f'{token_2}')).alias('spread')
).with_columns(
    pl.col('spread').rolling_mean(wind).alias('mean'),
    pl.col('spread').rolling_std(wind).alias('std')
).with_columns(
    ((pl.col('mean') - pl.col('spread')) / pl.col('std')).alias('z_score')
)

In [ ]:
df.tail(5)

In [ ]:
ct.replace(minute=0, second=0, microsecond=0) 

In [ ]:
ct.hour

In [ ]:
curr_time_bar